##Helper Methods

In [ ]:
# using my spreadsheet to get the attributes and real-world weights and then generate the participants
def generate_participants(n):
    df = pd.read_csv('/content/drive/MyDrive/css summer 2024/stanford/Fictional Participants - Sheet2.csv')
    attributes = {
        'age_group':
        {'categories': df.iloc[:, 0].dropna().tolist(), 'weights': df.iloc[:, 1].dropna().tolist()},
        'marital_status':
        {'categories': df.iloc[:, 2].dropna().tolist(), 'weights': df.iloc[:, 3].dropna().tolist()},
        'race':
        {'categories': df.iloc[:, 4].dropna().tolist(), 'weights': df.iloc[:, 5].dropna().tolist()},
        'sex':
        {'categories': df.iloc[:, 6].dropna().tolist(), 'weights': df.iloc[:, 7].dropna().tolist()},
        'education':
        {'categories': df.iloc[:, 8].dropna().tolist(), 'weights': df.iloc[:, 9].dropna().tolist()},
        'income':
        {'categories': df.iloc[:, 10].dropna().tolist(), 'weights': df.iloc[:, 11].dropna().tolist()},
        'mental_health':
        {'categories': df.iloc[:, 12].dropna().tolist(), 'weights': df.iloc[:, 13].dropna().tolist()},
        'political_affiliation':
        {'categories': df.iloc[:, 14].dropna().tolist(), 'weights': df.iloc[:, 15].dropna().tolist()}
    }
    data = {}
    for attribute, info in attributes.items():
        data[attribute] = np.random.choice(info['categories'], size=n, p=info['weights'])

    df = pd.DataFrame(data)

    return df, attributes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# get the system call string given the attributes of a persona
def generatePersonas(df):
  personas = []
  for i in range(len(df)):
    age = df.iloc[i, 0]
    marital_status = df.iloc[i, 1]
    race = df.iloc[i, 2]
    sex = df.iloc[i, 3]
    education = df.iloc[i, 4]
    income = df.iloc[i, 5]
    mental_health = df.iloc[i, 6]
    political_affiliation = df.iloc[i, 7]
    persona = f"You are a {age} year-old, {marital_status}, {race} {sex} with {education} making {income} per year, living in the United States. You have {mental_health} and your political affiliation is {political_affiliation}."
    personas.append(persona)
  return personas

In [ ]:
# given the set of messages and personas, making the calls
def calls(message, persona):
  user_content = """
  You are presented with a social media post. On a scale from 0 to 100, please indicate how you would feel,
  as a social media user, after seeing the social media post. Specifically, answer these questions:
  (1) if you would feel satisfied with yourself,
  (2) if you have a positive attitude towards yourself, and
  (3) if you would feel that you have a number of good qualities.
  Please just output three numbers, separated by a comma and a space, and nothing else.
  An example format is: "50, 50, 50" and nothing else.

  SOCIAL MEDIA POST:
  """
  user_content += message
  system_content = persona
  response = api_call(client, system_content, user_content)
  return response

In [ ]:
# just an API call
def api_call(client, system_content, user_content):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]
  )
  return response.choices[0].message.content

In [ ]:
def print_cols(df):
  cols = list(df.columns)
  print(cols)
  return

##Main Code

In [ ]:
my_api_key = "" # from PASCL's GPT-4 account

In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import openai
import os
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = my_api_key
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [ ]:
# main function cell
data = pd.read_csv('/content/drive/MyDrive/2024_07_08_Twitter_evaluations_Schoene_RatingLevel (1).csv').iloc[:, 2:3].dropna() # needed for preprocessing
data = data[:10] # just for testing
for index, row in data.iterrows():
  participants, attributes = generate_participants(5) # change number of participants per message here
  personas = generatePersonas(participants)
  #print(personas)    # see what personas are being generated
  i = 1
  for persona in personas:
    tweet = row['tweet']
    response = calls(tweet, persona)
    array = [int(item.strip()) for item in response.split(',')]
    name1 = "persona_" + str(i) + "_satisfaction"
    name2 = "persona_" + str(i) + "_quality"
    name3 = "persona_" + str(i) + "_positivity"
    data.at[index, name1] = array[0]
    data.at[index, name2] = array[1]
    data.at[index, name3] = array[2]
    i += 1

In [ ]:
print(data.shape)
print_cols(data)
print(data)

In [ ]:
data.to_csv('/content/drive/MyDrive/output.csv')